In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import os
import requests
import time
from pathlib import Path
from scipy import stats

In [2]:
# Set path to csv
monthly_rent_cost = Path("..\Resources\cleaned_rent_cost.csv")

# Reading into dataframes
rent_cost_df = pd.read_csv(monthly_rent_cost)

In [3]:
# home_values_df.set_index('Date', inplace=True)
rent_cost_df.head()

,Date,"Los Angeles, CA","Chicago, IL","Dallas, TX","Houston, TX","Washington, DC","Philadelphia, PA","Atlanta, GA","Phoenix, AZ","Riverside, CA",...,"Sebastian, FL","Idaho Falls, ID","Grand Junction, CO","Winchester, VA","Harrisonburg, VA","Manhattan, KS","Lawton, OK","California, MD","Cheyenne, WY","Oak Harbor, WA"
0,2015-01-31,1833.212831,1418.891001,1104.552997,1226.598136,1704.757792,1247.646914,1057.013776,952.601775,1360.118630,...,906.149690,604.752991,829.054590,1075.641997,890.643585,916.858487,695.409825,1124.231149,864.095756,865.923182
1,2015-02-28,1845.583162,1426.111814,1110.027098,1231.038381,1711.299329,1254.477544,1062.653419,967.517691,1363.492050,...,910.254445,608.855683,830.068647,1081.474518,897.619237,922.031163,697.931373,1134.692405,860.720567,868.879816
2,2015-03-31,1860.398826,1436.760649,1118.098082,1237.968825,1722.494861,1267.520439,1066.242647,974.900082,1368.107201,...,919.814529,614.393496,832.388966,1096.908316,897.817144,922.701802,704.178497,1141.577910,861.801386,877.337707
3,2015-04-30,1873.769750,1446.453042,1128.968868,1249.816502,1730.616446,1274.906633,1077.145991,987.374644,1375.332290,...,932.026063,620.753172,838.644583,1095.876558,883.666979,919.745471,704.949977,1129.014218,882.749069,895.289569
4,2015-05-31,1888.094972,1457.827846,1138.909149,1259.416606,1740.154076,1281.426448,1084.711468,987.860050,1382.974949,...,936.324352,622.644667,850.192195,1095.697324,888.448820,915.469885,706.721557,1134.298689,894.841896,909.581292


In [4]:
rent_cost_df.dtypes

Date                object
Los Angeles, CA    float64
Chicago, IL        float64
Dallas, TX         float64
Houston, TX        float64
                    ...   
Manhattan, KS      float64
Lawton, OK         float64
California, MD     float64
Cheyenne, WY       float64
Oak Harbor, WA     float64
Length: 194, dtype: object

In [5]:
# total dates in home values
total_dates = len(rent_cost_df)
total_dates

110

In [7]:
# total cities in rent_cost
total_cities = rent_cost_df.columns.unique()
total_cities

Index(['Date', 'Los Angeles, CA', 'Chicago, IL', 'Dallas, TX', 'Houston, TX',
       'Washington, DC', 'Philadelphia, PA', 'Atlanta, GA', 'Phoenix, AZ',
       'Riverside, CA',
       ...
       'Sebastian, FL', 'Idaho Falls, ID', 'Grand Junction, CO',
       'Winchester, VA', 'Harrisonburg, VA', 'Manhattan, KS', 'Lawton, OK',
       'California, MD', 'Cheyenne, WY', 'Oak Harbor, WA'],
      dtype='object', length=194)

In [8]:
# count total cities
len(total_cities)

194

In [9]:
# create a cities list
cities_list = total_cities.tolist()
cities_list.remove('Date')
len(cities_list)

193

In [10]:
# creating a dataframe for home values stats
rent_cost_summary = pd.DataFrame(rent_cost_df.describe())
rent_cost_summary

,"Los Angeles, CA","Chicago, IL","Dallas, TX","Houston, TX","Washington, DC","Philadelphia, PA","Atlanta, GA","Phoenix, AZ","Riverside, CA","Detroit, MI",...,"Sebastian, FL","Idaho Falls, ID","Grand Junction, CO","Winchester, VA","Harrisonburg, VA","Manhattan, KS","Lawton, OK","California, MD","Cheyenne, WY","Oak Harbor, WA"
count,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,...,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000
mean,2351.977544,1641.371366,1410.221298,1396.848239,1928.926778,1483.998083,1457.797448,1374.188152,1862.145273,1082.908007,...,1469.120590,867.278817,1086.377566,1340.061637,1162.073696,937.622652,801.438677,1330.460923,1019.191740,1315.279367
std,317.848406,159.242654,214.636894,137.598752,167.678790,173.711313,293.133125,319.250366,386.388860,166.606810,...,434.631154,211.031757,207.667047,226.397378,201.544636,45.142843,116.489968,171.586461,121.717210,254.313415
min,1833.212831,1418.891001,1104.552997,1226.598136,1704.757792,1247.646914,1057.013776,952.601775,1360.118630,854.439338,...,906.149690,604.752991,829.054590,1075.641997,883.666979,876.831657,691.354541,1124.231149,860.720567,865.923182
25%,2119.954837,1523.772265,1248.805009,1275.715258,1803.089010,1346.334010,1213.310015,1098.117848,1537.546143,943.610983,...,1131.918168,678.965129,896.629056,1153.583254,995.920782,907.079224,712.252818,1186.530135,916.752907,1152.897362
50%,2334.303472,1596.498515,1361.993948,1360.357385,1880.631595,1449.978310,1397.478911,1298.212082,1748.701545,1044.495629,...,1303.453894,812.762526,1040.266902,1265.263311,1092.016746,917.413124,742.683344,1259.933487,980.618930,1276.833412
75%,2621.065890,1718.650577,1618.585861,1521.917541,2056.065543,1634.100877,1784.838360,1770.827456,2275.823742,1241.913104,...,1881.224207,1106.003863,1281.345819,1547.772506,1294.564354,959.485606,868.975410,1517.506679,1134.700328,1522.750064
max,2903.100905,1991.389194,1780.568827,1657.738633,2272.513905,1811.343311,1929.490818,1862.364513,2497.532368,1381.245682,...,2408.635512,1215.269352,1461.174518,1775.828142,1563.799769,1054.829843,1049.821165,1673.013547,1258.960784,1800.188290


In [12]:
# finding the percent change between months in the home_values dataset for each city
rent_cost_set = rent_cost_df.set_index('Date')
rent_cost_numeric = rent_cost_set.apply(pd.to_numeric, errors = 'coerce')
rent_cost_pct_changes = pd.DataFrame(rent_cost_numeric.pct_change())
rent_cost_pct_changes

,"Los Angeles, CA","Chicago, IL","Dallas, TX","Houston, TX","Washington, DC","Philadelphia, PA","Atlanta, GA","Phoenix, AZ","Riverside, CA","Detroit, MI",...,"Sebastian, FL","Idaho Falls, ID","Grand Junction, CO","Winchester, VA","Harrisonburg, VA","Manhattan, KS","Lawton, OK","California, MD","Cheyenne, WY","Oak Harbor, WA"
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-28,0.006748,0.005089,0.004956,0.003620,0.003837,0.005475,0.005335,0.015658,0.002480,0.003853,...,0.004530,0.006784,0.001223,0.005422,0.007832,0.005642,0.003626,0.009305,-0.003906,0.003414
2015-03-31,0.008028,0.007467,0.007271,0.005630,0.006542,0.010397,0.003378,0.007630,0.003385,0.006412,...,0.010503,0.009095,0.002795,0.014271,0.000220,0.000727,0.008951,0.006068,0.001256,0.009734
2015-04-30,0.007187,0.006746,0.009723,0.009570,0.004715,0.005827,0.010226,0.012796,0.005281,0.008232,...,0.013276,0.010351,0.007515,-0.000941,-0.015761,-0.003204,0.001096,-0.011006,0.024307,0.020462
2015-05-31,0.007645,0.007864,0.008805,0.007681,0.005511,0.005114,0.007024,0.000492,0.005557,0.007431,...,0.004612,0.003047,0.013769,-0.000164,0.005411,-0.004649,0.002513,0.004681,0.013699,0.015963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-31,-0.001333,-0.003942,-0.003861,-0.003196,-0.000695,0.003395,-0.004140,-0.002582,0.005501,0.000318,...,0.041093,0.003100,0.010069,-0.012499,-0.008465,0.017065,0.028027,-0.007828,0.003573,0.035594
2023-11-30,-0.004530,-0.003939,-0.003516,-0.003289,-0.002411,0.000461,-0.006605,-0.004990,0.000146,0.001387,...,0.020295,-0.008643,-0.008916,0.008097,0.010396,0.006787,0.005252,-0.007981,-0.005023,-0.004846
2023-12-31,-0.003316,0.001688,-0.004926,-0.000929,-0.003120,-0.003078,-0.005492,-0.004047,-0.000396,0.002802,...,-0.000369,-0.008823,-0.011145,0.005581,0.024182,0.003300,-0.006369,0.000574,0.013004,0.010243


In [17]:
# Exporting to csv
rent_cost_pct_changes.to_csv('../Output/rc_pct_changes.csv', index=False)

In [15]:
# stats from pct_changes
rv_pct_changes_stats = pd.DataFrame(rent_cost_pct_changes.describe())
rv_pct_changes_stats

,"Los Angeles, CA","Chicago, IL","Dallas, TX","Houston, TX","Washington, DC","Philadelphia, PA","Atlanta, GA","Phoenix, AZ","Riverside, CA","Detroit, MI",...,"Sebastian, FL","Idaho Falls, ID","Grand Junction, CO","Winchester, VA","Harrisonburg, VA","Manhattan, KS","Lawton, OK","California, MD","Cheyenne, WY","Oak Harbor, WA"
count,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,...,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000
mean,0.004199,0.003134,0.004232,0.002732,0.002655,0.003434,0.005454,0.006108,0.005594,0.004426,...,0.008903,0.006409,0.005176,0.004553,0.005283,0.001311,0.003830,0.003695,0.003476,0.006569
std,0.004467,0.006345,0.005651,0.005017,0.005408,0.003910,0.005806,0.006734,0.005192,0.004370,...,0.018170,0.009090,0.008572,0.009252,0.017148,0.006939,0.009519,0.009092,0.005894,0.010318
min,-0.005695,-0.012192,-0.006528,-0.008350,-0.009106,-0.003841,-0.006605,-0.005158,-0.004138,-0.003588,...,-0.036747,-0.017630,-0.011662,-0.022445,-0.039412,-0.014831,-0.020324,-0.021827,-0.013053,-0.020641
25%,0.001789,-0.001858,0.000870,-0.000634,-0.001239,0.000249,0.001811,0.002120,0.002570,0.001123,...,-0.002948,-0.000220,-0.000769,-0.000941,-0.005185,-0.003701,-0.001109,-0.001124,-0.000705,0.000270
50%,0.003929,0.004383,0.003616,0.002525,0.002329,0.002924,0.004784,0.005376,0.004997,0.003896,...,0.008374,0.006751,0.004667,0.004790,0.004804,0.001293,0.002759,0.003766,0.003192,0.006597
75%,0.006184,0.007491,0.006794,0.005590,0.005511,0.005993,0.007896,0.008314,0.007103,0.007098,...,0.020295,0.012308,0.010326,0.010830,0.015486,0.005695,0.007153,0.009583,0.007231,0.013089
max,0.021690,0.017355,0.023199,0.019516,0.019734,0.013489,0.026248,0.032224,0.027439,0.015551,...,0.066924,0.031080,0.031071,0.034408,0.069383,0.019072,0.040626,0.025018,0.024307,0.035594


In [16]:
#finding the dates with the max and min values
max_dates = {}
for city in cities_list:
    max_value_date = rent_cost_pct_changes[city].idxmax()
    max_dates[city] = max_value_date

max_dates
# max_dates_df = pd.DataFrame(max_dates, index=['Max Date'])
# max_dates_df



{'Los Angeles, CA': '2021-08-31',
 'Chicago, IL': '2021-05-31',
 'Dallas, TX': '2021-06-30',
 'Houston, TX': '2021-06-30',
 'Washington, DC': '2021-06-30',
 'Philadelphia, PA': '2021-06-30',
 'Atlanta, GA': '2021-08-31',
 'Phoenix, AZ': '2021-06-30',
 'Riverside, CA': '2021-07-31',
 'Detroit, MI': '2021-05-31',
 'Seattle, WA': '2021-07-31',
 'Minneapolis, MN': '2017-03-31',
 'San Diego, CA': '2021-08-31',
 'Denver, CO': '2021-06-30',
 'Baltimore, MD': '2021-06-30',
 'St. Louis, MO': '2023-05-31',
 'Orlando, FL': '2021-06-30',
 'Charlotte, NC': '2021-08-31',
 'San Antonio, TX': '2021-07-31',
 'Portland, OR': '2021-06-30',
 'Sacramento, CA': '2021-07-31',
 'Pittsburgh, PA': '2017-02-28',
 'Cincinnati, OH': '2022-04-30',
 'Austin, TX': '2021-07-31',
 'Las Vegas, NV': '2021-06-30',
 'Kansas City, MO': '2021-06-30',
 'Columbus, OH': '2022-06-30',
 'Indianapolis, IN': '2021-06-30',
 'Cleveland, OH': '2021-07-31',
 'San Jose, CA': '2015-05-31',
 'Nashville, TN': '2021-07-31',
 'Virginia Beach